# PPO2 on Solo8 v2 Vanilla w/ Fixed Timestamp
Only use the time-based stopping criteria. This is more of a rudamentary test more than anything.

## Ensure that Tensorflow is using the GPU

In [1]:
import tensorflow as tf
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

Default GPU Device: /device:GPU:0


## Define Experiment Tags

In [2]:
TAGS = ['solov2vanilla', 'gpu', 'standing_task', 
        'unnormalized_actions']

# Import required libraries

In [3]:
from gym_solo.envs import solo8v2vanilla
from gym_solo.core import obs
from gym_solo.core import rewards
from gym_solo.core import termination as terms

import gym
import gym_solo

/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


## Parse CLI arguments and register w/ wandb

This experiment will be using the auto trainer to handle all of the hyperparmeter running

In [4]:
from auto_trainer import params
import auto_trainer

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Give the robot a total of 10 seconds simulation time to learn how to stand.

In [5]:
episode_length = 2 / solo8v2vanilla.Solo8VanillaConfig.dt
episode_length

2000.0

Create a basic config

In [6]:
config = params.WandbParameters().parse()

config.episodes = 12500
config.episode_length = episode_length

config.num_workers = 6
config.eval_frequency = 50
config.eval_episodes = 3
config.fps = 15

# Create a 3 second gif
config.eval_render_freq = int(config.episode_length / (3 * config.fps))

config

Namespace(algorithm='PPO2', episode_length=2000.0, episodes=12500, eval_episodes=3, eval_frequency=50, eval_render_freq=44, fps=15, num_workers=6, policy='MlpPolicy')

In [7]:
config, run = auto_trainer.get_synced_config(config, TAGS)
config

wandb: Currently logged in as: agupta231 (use `wandb login --relogin` to force relogin)
/usr/local/lib/python3.7/dist-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


{'episodes': 12500, 'episode_length': 2000.0, 'policy': 'MlpPolicy', 'algorithm': 'PPO2', 'num_workers': 6, 'eval_episodes': 3, 'eval_frequency': 50, 'eval_render_freq': 44, 'fps': 15}

Add the following inputs to the robot / environment:

**Observations**
- TorsoIMU
- Motor encoder current values

**Reward**
- How upright the TorsoIMU is. Valued in $[-1, 1]$

**Termination Criteria**
- Terminate after $n$ timesteps

Note that the autotrainer requires that the training environment be a `VecEnv` and the testing environment be a standard `gym.Env` for multi-processing.

For us personally, we find that the easiest way to handle this is to create a Stable Baselines `VecEnv` generator (example can be found [here](https://stable-baselines.readthedocs.io/en/master/guide/examples.html#multiprocessing-unleashing-the-power-of-vectorized-environments)) and use that to generate both the training and testing environments.

We also like to link our generator with our W&B config so that we can dynamically change the environments based from the web interface. 

A full example can be seen below:

In [8]:
def make_env(length):
    def _init():
        env_config = solo8v2vanilla.Solo8VanillaConfig()
        env = gym.make('solo8vanilla-v0', config=env_config, 
                       normalize_actions=False)

        env.obs_factory.register_observation(obs.TorsoIMU(env.robot))
        env.obs_factory.register_observation(obs.MotorEncoder(env.robot))

        env.reward_factory.register_reward(1, rewards.UprightReward(env.robot))
        env.termination_factory.register_termination(terms.TimeBasedTermination(length))
        return env
    return _init

### Create the Envs
Import the desired vectorized env

In [9]:
from stable_baselines.common.vec_env import SubprocVecEnv
from stable_baselines.common.vec_env import VecNormalize

Create training & testing environments

In [10]:
train_env = SubprocVecEnv([make_env(config.episode_length) 
                           for _ in range(config.num_workers)])
# train_env = VecNormalize(train_env, clip_reward = 1.)

test_env = make_env(config.episode_length)()

/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


## Learning
And we're off!

In [ ]:
model, config, run = auto_trainer.train(train_env, test_env, config, TAGS, 
                                        log_freq=1000, full_logging=False, run=run)





Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




-------------------------------------
| approxkl           | 0.0026667363 |
| clipfrac           | 0.02311198   |
| explained_variance | 0.0104       |
| fps                | 962          |
| n_updates          | 1            |
| policy_entropy     | 17.029896    |
| policy_loss        | -0.008255407 |
| serial_timesteps   | 128          |
| time_elapsed       | 0.000259     |
| total_timesteps    | 768          |
| value_loss         | 16.26643     |
-------------------------------------



wandb: WARNING Step cannot be set when using syncing with tensorboard. Please log your step values as a metric such as 'global_step'


-------------------------------------
| approxkl           | 0.001689617  |
| clipfrac           | 0.008463541  |
| explained_variance | 0.802        |
| fps                | 2661         |
| n_updates          | 1000         |
| policy_entropy     | 20.518154    |
| policy_loss        | -0.004469267 |
| serial_timesteps   | 128000       |
| time_elapsed       | 408          |
| total_timesteps    | 768000       |
| value_loss         | 6.4817324    |
-------------------------------------
-------------------------------------
| approxkl           | 0.0015881007 |
| clipfrac           | 0.010091146  |
| explained_variance | 0.802        |
| fps                | 2757         |
| n_updates          | 2000         |
| policy_entropy     | 23.67096     |
| policy_loss        | -0.00787298  |
| serial_timesteps   | 256000       |
| time_elapsed       | 821          |
| total_timesteps    | 1536000      |
| value_loss         | 0.24593358   |
-------------------------------------
------------